In [ ]:
%matplotlib inline
from datetime import date, datetime, timedelta, time
import pandas as pd
import seaborn
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import scipy

In [ ]:
from simlightcurve.curves import GaussExp as Gred
from simlightcurve.solvers import find_rise_t, find_peak
from simtransient.utils import mahalanobis_sq, build_covariance_matrix
from scipy.stats import norm, multivariate_normal
import scipy.optimize as op
import emcee
from collections import OrderedDict

In [ ]:
seaborn.set_context('poster')
current_palette = seaborn.color_palette()

In [ ]:
ndim=2

param_variances = OrderedDict()
param_variances['x']=3
param_variances['y']=0.5
print "Variances:", param_variances
param_correlations = {('x','y'):-0.9}

cov = build_covariance_matrix(param_variances, param_correlations)
print "Det:", np.linalg.det(cov)
print "S:", np.linalg.svd(cov)[1]
cov

In [ ]:
param_variances.values()[0]

In [ ]:

true_mean = np.random.random(size=2)
print "True Mean: ", true_mean
rv = multivariate_normal(mean=true_mean, cov=cov)
plotwidth=2
scale = plotwidth*param_variances['x']
x = np.linspace(true_mean[0]-scale,
                true_mean[0]+scale,
                100)
y = np.linspace(true_mean[1]-scale,
                true_mean[1]+scale,
                100)

grid = np.dstack(np.meshgrid(x,y))
# print x[0],x[1]
# y = np.sqrt(x[0]**2+x[1]**2)

seaborn.set_palette('husl')
cmap = plt.cm.get_cmap('afmhot')
plt.contourf(x,y,rv.pdf(grid),50,cmap=cmap)
plt.ylabel('i')
plt.xlabel('j')
plt.axvline(true_mean[0], ls=':')
plt.axhline(true_mean[1], ls=':')
plt.colorbar()

n_samples = 5
data = np.atleast_2d(rv.rvs(n_samples))
# print "data:", data
print "datashape", data.shape
plt.scatter(data[:,0],data[:,1])
# plt.scatter(data[0,:],data[1,:])

In [ ]:
def lnprob(mu,x,icov):
    diff = x-mu
    return -0.5*(np.sum(mahalanobis_sq(icov,diff)))

In [ ]:
icov = np.linalg.inv(cov)
c = np.sqrt(((2*np.pi)**ndim)*np.linalg.det(cov))

In [ ]:
test_position = true_mean+np.asarray((1.1,1.1))
prob = (1./c)*np.exp(lnprob(true_mean, test_position, icov))

In [ ]:
scipy_prob = rv.pdf(test_position)
print prob, scipy_prob
print scipy_prob - prob

In [ ]:
neg_likelihood = lambda *args: -lnprob(*args)
mean_guess=np.asarray((0,0))
result = op.minimize(neg_likelihood, mean_guess, args=(data,icov))
print true_mean
print result.x

In [ ]:
cmap = plt.cm.get_cmap('afmhot')
plt.contourf(x,y,rv.pdf(grid),25,cmap=cmap)
plt.ylabel('i')
plt.xlabel('j')
plt.axvline(true_mean[0], ls=':')
plt.axhline(true_mean[1], ls=':')
plt.colorbar()

plt.scatter(data[:,0],data[:,1])
plt.scatter(*result.x, marker='x', s=100, c='red', linewidths=3)

In [ ]:
ndim = 2  # number of parameters in the model
nwalkers = 100  # number of MCMC walkers
nsteps = 1500  # number of MCMC steps to take

mean_init = [data[0] + 1e-5*np.random.randn(ndim) for i in range(nwalkers)]
# pos = [0.5 + 1e-4*np.random.randn(ndim) for i in range(nwalkers)]
# set theta near the maximum likelihood, with 
# sampler.reset()
# sampler = emcee.EnsembleSampler(nwalkers, ndim, log_likelihood, args=(data,))
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob, args=(data,icov),a=4,threads=4)
sampler.reset()
_=sampler.run_mcmc(mean_init, nsteps)
seaborn.set_palette('deep')
seaborn.set_style('darkgrid')
for walker in sampler.chain:
    plt.plot(walker)

In [ ]:
acorr = np.ceil(np.max(sampler.get_autocorr_time()))
nburn = np.ceil(acorr*2.5)
print acorr, nburn

In [ ]:
acceptance = np.median(sampler.acceptance_fraction)
print acceptance
sampler.chain.shape

In [ ]:
thinned = sampler.chain[:,::acorr,:]
thinned = thinned.ravel().reshape(-1,ndim)

In [ ]:
plotchain=sampler.chain
varindex = 1
plt.subplot(2,1,1)
for walker in plotchain[:,:,varindex]:
    plt.plot(walker)
plt.axvline(nburn, ls=':', color='k')
print "Acorr",sampler.get_autocorr_time()
print "Acceptance",acceptance
plt.subplot(2,1,2)
for walker in plotchain[:,::acorr,varindex]:
    plt.plot(walker)
plt.axvline(nburn/acorr, ls=':', color='k')

In [ ]:
seaborn.set_style('dark')
import triangle
_=triangle.corner(thinned,
                  truths=true_mean,
                  plot_contours=False,
                 quantiles=[0.16, 0.5, 0.84],
                 )